In [1]:
# Now I want write a test script to test all testcases in testcases folder
# The testcases in such format
# <name>.sy source code
# <name>.in input
# <name>.out std output

# compiler path: ./build/SYSY-compiler
# compiler command: ./build/SYSY-compiler <name>.sy
# I need you to run the compiler and get the output llvm ir code, which will be put in <name>.ll 
# If the compiler doesn't end with 0, keep a record of the trouble file name in CE_list
# If the compiler ends with 0, run the llvm ir code with lli and get the output, which will be put in <name>.outq
# Compare it with the std output <name>.out ,if is't not that same ,keep a record of the trouble file name in WA_list
# Remember to redirect the input and output of lli

import os
import sys
import subprocess
import filecmp
import difflib

compiler_path = "./build/SYSY-compiler"
test_folder="./testcases"

pass_args=["--mem2reg","--ece","--pre"]

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".sy"):
            test_list.append(os.path.join(root, file))

CE_list = []
LLI_FAIL_list = []
WA_list = []
AC_list = []
TLE_list = []
Time_Out=[]

Bad_test = []

for test in test_list:
    if test.endswith(".sy"):
        # add test_args to list
        compile_args=[compiler_path, test]
        for arg in pass_args:
            compile_args.append(arg)
        try:
            ret = subprocess.run(compile_args,timeout=60)
        except subprocess.TimeoutExpired:
            Time_Out.append(test)
            continue
        if ret.returncode != 0:
            CE_list.append(test)
            # print("Compiler Error")
            continue
        # run lli
        ll_file = test+".ll"
        out_file = test[:-2]+"out"
        # maybe theres no input file
        
        if not os.path.exists(test[:-2]+"in") and not os.path.exists(out_file):
            Bad_test.append(test)
            continue
        
        # Kill program run over 5s, give it a TLE
        try:
            if not os.path.exists(test[:-2]+"in"):
                ret = subprocess.run(["lli","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=3)
            else:
                ret = subprocess.run(["lli","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=open(test[:-2]+"in").read().encode(), timeout=3)
        except subprocess.TimeoutExpired:
            print("Timeout Error")
            TLE_list.append(test)
            continue
            
        # compare the output
        # Merge The reture Value and stdout
        
        dump_str=ret.stdout.decode()
        # remove whitesspace in the end
        # dump_str=dump_str.rstrip()
        if dump_str and not dump_str.endswith('\n'):
            dump_str += "\n"
        dump_str += str(ret.returncode) + "\n"
        std_output=open(out_file).read()
        diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
        if(len(list(diff))!=0):
            # print("Wrong Answer")
            WA_list.append(test)
            continue
        # print("Accepted")
        AC_list.append(test)
        
print("Compiler Error: Total: "+str(len(CE_list)))
print("Runtime Error: Total: "+str(len(LLI_FAIL_list)))
print("Timeout Error: Total: "+str(len(TLE_list)))
print("Wrong Answer: Total: "+str(len(WA_list)))
print("TimeOut Function:"+str(len(Time_Out)))
print("Bad Test: Total: "+str(len(Bad_test)))
print("Accepted: Total: "+str(len(AC_list)))

Compiler Error: Total: 0
Runtime Error: Total: 0
Timeout Error: Total: 0
Wrong Answer: Total: 0
TimeOut Function:0
Bad Test: Total: 62
Accepted: Total: 148


In [2]:
k=0
if len(WA_list) > k:
    print(WA_list[k])
    # move the file to ./function_test/test.c
    os.system("cp "+WA_list[k]+" ./function_test/test.c")
    os.system("cp "+WA_list[k][:-2]+"in"+" ./function_test/test.in")
    os.system("cp "+WA_list[k][:-2]+"out"+" ./function_test/test.out")

./testcases/69_expr_eval.sy


In [2]:
k=0
if len(Time_Out) > k:
    print(Time_Out[k])
    # move the file to ./function_test/test.c
    os.system("cp "+Time_Out[k]+" ./function_test/test.c")
    os.system("cp "+Time_Out[k][:-2]+"in"+" ./function_test/test.in")
    os.system("cp "+Time_Out[k][:-2]+"out"+" ./function_test/test.out")

./testcases/29_long_line.sy


cp: cannot stat './testcases/29_long_line.in': No such file or directory


In [10]:
import os
import shutil

current_dir = os.getcwd()
target_folder=os.path.join(current_dir,"include")
# 遍历当前目录下的所有文件和文件夹
for root, dirs, files in os.walk(current_dir):
    if root=='/mnt/e/miniC-compiler/ir/opt':
        os.makedirs(os.path.join(target_folder,"ir/opt"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"ir/opt")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"ir/opt"))
    if root=='/mnt/e/miniC-compiler/backend':
        os.makedirs(os.path.join(target_folder,"backend"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"backend")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"backend"))
    if root=='/mnt/e/miniC-compiler/ir/Analysis':
        os.makedirs(os.path.join(target_folder,"ir/Analysis"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"ir/Analysis")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"ir/Analysis"))
    if root=='/mnt/e/miniC-compiler/yacc': 
        os.makedirs(os.path.join(target_folder,"yacc"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp") or file.endswith(".hh"):
                if not os.path.exists(os.path.join(target_folder,"yacc")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"yacc"))
    if root=='/mnt/e/miniC-compiler/lib':
        os.makedirs(os.path.join(target_folder,"lib"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"lib")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"lib"))


In [2]:
k=0
if len(CE_list) > k:
    print(CE_list[k])
    # move the file to ./function_test/test.c
    os.system("cp "+CE_list[k]+" ./function_test/test.c")
    os.system("cp "+CE_list[k][:-2]+"in"+" ./function_test/test.in")
    os.system("cp "+CE_list[k][:-2]+"out"+" ./function_test/test.out")

./testcases/55_sort_test1.sy


cp: cannot stat './testcases/55_sort_test1.in': No such file or directory
